In [9]:
%matplotlib inline
from __future__ import print_function
import collections
import math
import numpy as np
import os
import random
import tensorflow as tf
import zipfile
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
from sklearn.manifold import TSNE

In [10]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urlretrieve(url=url+filename, filename=filename)
    else:
        print ('%s file already present. Skipping download.' %filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified %s' % filename)
    else:
        print(statinfo.st_size)
        raise Exception('Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

filename = maybe_download('text8.zip', 31344016)

text8.zip file already present. Skipping download.
Found and verified text8.zip


In [11]:
def read_data(filename):
    """Extract the first file enclosed in a zip file as a list of words"""
    with zipfile.ZipFile(filename) as f:
        words = tf.compat.as_str (f.read(f.namelist()[0])).split()
    return words

words = read_data(filename)
print('Data size %d' % len(words))

Data size 17005207


In [12]:
vocabulary_size = 50000

def build_dataset(words):
    count = [['UNK', -1]] # count is a list of list
    collections.Counter()
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary) # Dictionary 1-1 karke construct ho rha hai..
                                           # Dictionary[first word in Count 'UNK'] = 1
                                           # Dictionary[second word in Count 'The'] = 2
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0
            unk_count = unk_count + 1
        data.append(index)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10])
del words  # Hint to reduce memory.

Most common words (+UNK) [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)]
Sample data [5241, 3082, 12, 6, 195, 2, 3137, 46, 59, 156]


In [16]:
data_index = 0

def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size%num_skips == 0
    assert num_skips <= 2*skip_window
    
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    
    span = 2 * skip_window + 1
    buffer = collections.deque(maxlen = span)
    
    for _ in range(span):
        buffer.append (data[data_index])
        data_index = data_index + 1
        
    for i in range(batch_size // num_skips):
        target = skip_window # target label at the center of the buffer
        targets_to_avoid = [target]
        for j in range(num_skips):
            # While loop makes sure, we pick unique label each time
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            
            batch[i*num_skips + j] = buffer[skip_window]
            labels[i*num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    return batch, labels


print ('data', [reverse_dictionary[x] for x in data[:8]])

"""First things first SKIP here is JUMP, not skip...
    For the sentence, anarchism oriented as a term of abuse first,
    skip_window = aage piche kitne ka window dekhna hai.. 
    num_skips = a subset of skip_window, out of 2*skip_window words forward&backword, random num_skip words pick karo
    
    So if skip_window is 2, aage k 2 words and piche k 2 words are of interest
    and say num_skips is 2, then 4 possible aage-piche words me se, select DISTINCT 2 words
    and say num_skips is 4, then 4 possible aage-piche words me se, select DISTINCT 4 words
    
    num_skips cannot be greater than 2*skip_window"""
for num_skips, skip_window in [(2, 1), (4, 2)]:
    data_index = 0
    batch, labels = generate_batch(batch_size=8, num_skips=num_skips, skip_window=skip_window)
    print('\nwith num_skips = %d and skip_window = %d:' % (num_skips, skip_window))
    print('    batch:', [reverse_dictionary[bi] for bi in batch])
    print('    labels:', [reverse_dictionary[li] for li in labels.reshape(8)])

data ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first']

with num_skips = 2 and skip_window = 1:
    batch: ['originated', 'originated', 'as', 'as', 'a', 'a', 'term', 'term']
    labels: ['anarchism', 'as', 'originated', 'a', 'term', 'as', 'a', 'of']

with num_skips = 4 and skip_window = 2:
    batch: ['as', 'as', 'as', 'as', 'a', 'a', 'a', 'a']
    labels: ['anarchism', 'term', 'a', 'originated', 'as', 'originated', 'of', 'term']
